# Ejemplo

d) Select five assets from the market, calculate their returns, and, with this, calculate the expected return and
the covariance matrix. Once these parameters are obtained, the optimization problem will be solved, and
investment recommendations will be generated. Remember to verify that each of the assets has sufficient
liquidity.

## Código

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from IPython.display import display, Markdown
from ipywidgets import interact, FloatSlider

## Fórmula para $w$ que optimiza el ratio de Sharpe
$$
w = \frac{\Sigma^{-1} (\mu - r_f)}{1^T \Sigma^{-1} (\mu - r_f)}
$$